# **KUBIG 24S NLP Basic Team 4**

## **사투리 번역기 구현**

### 19기 정종락

# **Import Libraries, Modules, and Dataset**

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/data/'
columns_to_load = ['standard_form', 'dialect_form']

Mounted at /content/drive


In [3]:
kw_train = pd.read_parquet(path + 'kw_train.parquet',
                           columns = columns_to_load
                           )

## **Dataset**

In [ ]:
kw_train.shape

(1573237, 2)

In [ ]:
kw_train.head(10)

,standard_form,dialect_form
0,난 네가 내 가장 친한 친구였으면 좋겠단 말이야.,난 니가 내 채고 친한 친구였으문 좋겠단 말이야.
1,내 마음을 받아달라는 거 아니야.,내 마옴을 받아달라는 거 아니야.
2,내 마음을 네가 알아달라는 것 뿐이야.,내 마옴을 니가 알아달라는 것 뿐이야.
3,내가 네 옆에 있다는 것을 기억해 주면 돼.,내가 니 옆에 있다는 것을 기억해 주면 돼.
4,내가 이런 고백을 했다구 날 멀리 하진 마.,내가 이딴 고백을 했다구 날 멀리 하진 마.
5,물론 내가 그러라고 내버려두진 않겠지만.,물론 내가 그러라고 내버려두진 않겠지만.
6,&name1&야 &name1& 난 널 잃고 싶지 않아.,&name1&야 &name1& 난 널 잃고 싶지 않아.
7,걱정하지마.,극정하지마.
8,그런 일은 없어.,그른 일은 없어.
9,내가 널 잃지 않을 거니까.,내가 널 잃지 않을 거니까.


In [ ]:
kw_train.tail(10)

,standard_form,dialect_form
1573227,고맙고 좋은 분이셔.,고맙고 좋은 분이셔.
1573228,팀장님이랑 함께 있으면 벌써 봄이 온 거 같애.,팀장님이랑 함께 있으문 벌써 봄이 온 거 같애.
1573229,너도 팀장님 좋아하는구나?,너도 팀장님 좋아하는구나?
1573230,이런 내 마음이 사랑이 아니라 해도 팀장님과 다시 시작해 보고 싶어.,이딴 내 마음이 사랑이 아니라 해도 팀장님과 다시 시작해 보고 싶어.
1573231,&name2& 씨랑 헤어진 지도 얼마 안 됐는데 너무 이상하지?,&name2& 씨랑 헤어진 지도 얼매 안 됐는데 너머 이상하지?
1573232,하나도 안 이상하다.,한나도 안 이상하다.
1573233,자연스럽고 당연한 거야.,자연스럽고 당연한 기야.
1573234,어둡고 깜깜할 때 켜진 촛불 하나가 수백 개 전구보다 더 밝을 수도 있지.,어둡고 깜깜할 때 켜진 쵯불 한나가 수백 개 전구보다 더 밝을 수도 있지.
1573235,너 힘들 때 옆에 있어준 사람한테 마음 주는 거야 당연하지.,너 힘들 때 옆에 있어준 사램한테 마옴 주는 거야 당연하지.
1573236,정말 그럴까?,증말 그럴까?


# **Model**

## **표준어 - 강원도, KoBART - Batch Size 32, epoch 5**

In [4]:
kw_train_sampled = kw_train.sample(n = 100000, random_state = 42)

In [5]:
kw_train_sampled

,standard_form,dialect_form
853337,아저씨 공 좀 차주세요.,아이씨 공 좀 차주세요.
76928,어젠 고마웠다.,어젠 고마웠다.
642740,맞아 맞아.,맞아 맞아.
57150,신뢰만으로 사랑하지 않으면서 결혼하려고 했습니다.,신뢰만으로 사랑하지 않으면서 겨론하려고 했습니다.
1420922,당신은 가만있어.,당신은 가만있어.
...,...,...
248116,그걸 죽 돌려 나눠 갖도록 하세요 서류 따위.,그글 죽 돌려 나눠 갖도록 하세요 서류 따위.
269181,뭐야?,뭐야?
397372,왜?,왜?
1490824,무지막지로 달려들면 무섭지.,무닥떼기로 달려들면 무섭지.


In [6]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터셋 로드
df = kw_train_sampled.copy()

# 필요한 전처리 (예: 결측값 제거 등)
df = df.dropna()

# 학습 및 검증 셋 분리
train_df, val_df = train_test_split(df, test_size = 0.2, random_state = 42)

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

# 토큰화 함수 정의
def tokenize_data(texts):
    return tokenizer(texts, padding = "max_length",
                     max_length = 128, truncation = True, return_tensors = "pt")

# 토큰화 수행
train_encodings = tokenize_data(train_df['standard_form'].tolist())
train_labels = tokenize_data(train_df['dialect_form'].tolist())

val_encodings = tokenize_data(val_df['standard_form'].tolist())
val_labels = tokenize_data(val_df['dialect_form'].tolist())

import torch
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

# 데이터셋 생성
train_dataset = TranslationDataset(train_encodings, train_labels)
val_dataset = TranslationDataset(val_encodings, val_labels)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [7]:
from transformers import BartForConditionalGeneration, AdamW

model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')
optimizer = AdamW(model.parameters(), lr = 2e-5)

from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # 데이터 준비
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 모델 예측 및 손실 계산
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 진행 상황 출력
        loop.set_description(f'Epoch {epoch + 1}')
        loop.set_postfix(loss=loss.item())

    # 검증 단계 (옵션)
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

    print(f'Validation Loss: {val_loss / len(val_loader)}')

# 모델과 토크나이저 저장 경로
model_save_path = '/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/model/translation_model_st_kw'
tokenizer_save_path = '/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/model/translation_tokenizer_st_kw'

# 모델 저장
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/2500 [00:00<?, ?it/s]<ipython-input-6-efb7ad14bdaf>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-6-efb7ad14bdaf>:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels['input_ids'][idx])
Epoch 1: 100%|██████████| 2500/2500 [09:14<00:00

Validation Loss: 0.05763190100193024


Epoch 2: 100%|██████████| 2500/2500 [09:13<00:00,  4.52it/s, loss=0.0479]


...

Validation Loss: 0.032373202794790265


Epoch 5: 100%|██████████| 2500/2500 [09:12<00:00,  4.52it/s, loss=0.0132]


Validation Loss: 0.031231058804690837


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}


('/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/model/translation_tokenizer_st_kw/tokenizer_config.json',
 '/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/model/translation_tokenizer_st_kw/special_tokens_map.json',
 '/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/model/translation_tokenizer_st_kw/tokenizer.json')

In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

model = BartForConditionalGeneration.from_pretrained(model_save_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_save_path)
model.to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [10]:
# 입력 문장
input_sentence = "하나도 안 이상하다."

# 토큰화
input_ids = tokenizer.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

하나도 안 이상하다.


In [ ]:
# 입력 문장
input_sentence = "어둡고 깜깜할 때 켜진 촛불 하나가 수백 개 전구보다 더 밝을 수도 있지."

# 토큰화
input_ids = tokenizer.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

어둡고 깜깜할 때 켜진 촛불 하나가 수백 개 전구보다 더 밝을 수도 있지.


In [ ]:
# 입력 문장
input_sentence = "너 힘들 때 옆에 있어준 사람한테 마음 주는 거야 당연하지. 정말 그럴까?	"

# 토큰화
input_ids = tokenizer.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

너 힘들 때 옆에 있어준 사람한테 마음 주는 거야 당연하지. 증말 그럴까?
